In [1]:
import warnings
warnings.filterwarnings(action='ignore')
%config Completer.use_jedi = False
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np
import pandas as pd

Instructions for updating:
non-resource variables are not supported in the long term


지문 읽고 주제 분류하기

다음과 같은 데이터를 데이터프레임으로 저장한다. 입력 데이터는 음식 및 스포츠 관련 지문으로 구성되어 있다.

In [2]:
paragraph_dict_list = [
    {'paragraph': 'Dishplace is located in sunnyvale downtown there is parking around the area but it can be difficult to find during peak business hours my sisters and i came to this place for dinner on a weekday they were really busy so i highly recommended making reservations unless you have the patience to wait', 'category': 'food'},
    {'paragraph': 'Sevice can be slower during busy hours but our waiter was courteous and help gave some great entree recommendations', 'category': 'food'},
    {'paragraph': 'Portions are huge both french toast and their various omelettes are really good their french toast is probably 1.5x more than other brunch places great place to visit if you are hungry and dont want to wait 1 hour for a table', 'category': 'food'},
    {'paragraph': 'We started with apps going the chicken and waffle slides and chicken nachos the sliders were amazing and the nachos were good too maybe by themselves the nachos would have scored better but after those sliders they were up against some tough competition', 'category': 'food'},
    {'paragraph': 'The biscuits and gravy was too salty two people in my group had the gravy and all thought it was too salty my hubby ordered a side of double egg and it was served on two small plates who serves eggs to one person on separate plates we commented on that when it was delivered and even the server laughed and said she doesnt know why the kitchen does that presentation of food is important and they really missed on this one', 'category': 'food'},
    {'paragraph': 'The garlic fries were a great starter (and a happy hour special) the pancakes looked and tasted great and were a fairly generous portion', 'category': 'food'},
    {'paragraph': 'Our meal was excellent i had the pasta ai formaggi which was so rich i didnt dare eat it all although i certainly wanted to excellent flavors with a great texture contrast between the soft pasta and the crisp bread crumbs too much sauce for me but a wonderful dish', 'category': 'food'},
    {'paragraph': 'What i enjoy most about palo alto is so many restaurants have dog-friendly seating outside i had bookmarked italico from when they first opened about a 1.5 years ago and was jonesing for some pasta so time to finally knock that bookmark off', 'category': 'food'},
    {'paragraph': 'The drinks came out fairly quickly a good two to three minutes after the orders were taken i expected my iced tea to taste a bit more sweet but this was straight up green tea with ice in it not to complain of course but i was pleasantly surprised', 'category': 'food'},
    {'paragraph': 'Despite the not so good burger the service was so slow the restaurant wasnt even half full and they took very long from the moment we got seated to the time we left it was almost 2 hours we thought that it would be quick since we ordered as soon as we sat down my coworkers did seem to enjoy their beef burgers for those who eat beef however i will not be returning it is too expensive and extremely slow service', 'category': 'food'},
    {'paragraph': 'The four reigning major champions simona halep caroline wozniacki angelique kerber and defending us open champion sloane stephens could make a case for being the quartet most likely to succeed especially as all but stephens has also enjoyed the no1 ranking within the last 14 months as they prepare for their gruelling new york campaigns they currently hold the top four places in the ranks', 'category': 'sports'},
    {'paragraph': 'The briton was seeded nn7 here last year before a slump in form and confidence took her down to no46 after five first-round losses but there have been signs of a turnaround including a victory over a sub-par serena williams in san jose plus wins against jelena ostapenko and victoria azarenka in montreal. konta pulled out of new haven this week with illness but will hope for good things where she first scored wins in a major before her big breakthroughs to the semis in australia and wimbledon', 'category': 'sports'},
    {'paragraph': 'Stephens surged her way back from injury in stunning style to win her first major here last year—and ranked just no83 she has since proved what a big time player she is winning the miami title via four fellow major champions then reaching the final at the french open back on north american hard courts she ran to the final in montreal only just edged out by halep she has also avoided many of the big names in her quarter—except for wild card azarenka as a possible in the third round', 'category': 'sports'},
    {'paragraph': 'When it came to england chances in the world cup it would be fair to say that most fans had never been more pessimistic than they were this year after enduring years of truly dismal performances at major tournaments – culminating in the 2014 event where they failed to win any of their three group games and finished in bottom spot those results led to the resignation of manager roy hodgson', 'category': 'sports'},
    {'paragraph': 'The team that eliminated russia – croatia – also improved enormously during the tournament before it began their odds were 33/1 but they played with real flair and star players like luka modric ivan rakitic and ivan perisic showed their quality on the world stage having displayed their potential by winning all three of their group stage games croatia went on to face difficult tests like the semi-final against england', 'category': 'sports'},
    {'paragraph': 'The perseyside outfit finished in fourth place in the premier league table and without a trophy last term after having reached the champions league final before losing to real madrid', 'category': 'sports'},
    {'paragraph': 'Liverpool fc will return to premier league action on saturday lunchtime when they travel to leicester city in the top flight as they look to make it four wins in a row in the league', 'category': 'sports'},
    {'paragraph': 'Alisson signed for liverpool fc from as roma this summer and the brazilian goalkeeper has helped the reds to keep three clean sheets in their first three premier league games', 'category': 'sports'},
    {'paragraph': 'But the rankings during that run-in to new york hid some very different undercurrents for murray had struggled with a hip injury since the clay swing and had not played a match since losing his quarter-final at wimbledon and he would pull out of the us open just two days before the tournament began—too late however to promote nederer to the no2 seeding', 'category': 'sports'},
    {'paragraph': 'Then came the oh-so-familiar djokovic-nadal no-quarter-given battle for dominance in the third set there were exhilarating rallies with both chasing to the net both retrieving what looked like winning shots nadal more than once pulled off a reverse smash and had his chance to seal the tie-break but it was djokovic serving at 10-9 who dragged one decisive error from nadal for a two-sets lead', 'category': 'sports'}
]
df = pd.DataFrame(paragraph_dict_list)
df

,paragraph,category
0,Dishplace is located in sunnyvale downtown the...,food
1,Sevice can be slower during busy hours but our...,food
2,Portions are huge both french toast and their ...,food
3,We started with apps going the chicken and waf...,food
4,The biscuits and gravy was too salty two peopl...,food
5,The garlic fries were a great starter (and a h...,food
6,Our meal was excellent i had the pasta ai form...,food
7,What i enjoy most about palo alto is so many r...,food
8,The drinks came out fairly quickly a good two ...,food
9,Despite the not so good burger the service was...,food


데이터 전처리

LSTM 모델이 입력 데이터를 처리할 수 있도록 수치값으로 변경한다.  
텍스트인 입력값을 수치로 변환하기 위해서 지문에 사용된 모든 단어들을 모아서 중복을 제거한 후 단어 리스트를 만든다.

In [3]:
# 중복을 제거해서 지문에 사용된 단어를 모으기 위해서 빈 set을 선언한다.
results = set()

# 데이터프레임의 paragraph열(Series)을 모두 소문자로 변환한 후 공백을 경계로 나눠서 set에 저장한다.
# print(type(results)) # <class 'set'>
# print(type(df)) # <class 'pandas.core.frame.DataFrame'>
# print(type(df.paragraph)) # <class 'pandas.core.series.Series'>
# print(type(df.paragraph.str)) # <class 'pandas.core.strings.accessor.StringMethods'>
# print(type(df.paragraph.str.lower())) # <class 'pandas.core.series.Series'>
# print(type(df.paragraph.str.lower().str)) # <class 'pandas.core.strings.accessor.StringMethods'>
# print(type(df.paragraph.str.lower().str.split())) # <class 'pandas.core.series.Series'>
df.paragraph.str.lower().str.split().apply(results.update)
print(len(results))
print(results)

537
{'their', 'sub-par', 'wonderful', 'tasted', 'why', 'dinner', 'enjoy', 'dominance', 'courts', 'waiter', 'group', 'year—and', 'were', 'illness', 'clean', 'serves', 'commented', 'look', 'ai', 'business', 'victoria', 'win', 'quarter-final', 'time', 'although', 'three', 'a', 'of', 'halep', 'dismal', 'much', 'than', 'as', 'once', 'around', 'hours', 'some', 'roma', 'parking', 'served', 'places', 'exhilarating', 'if', 'left', 'rich', 'didnt', 'proved', 'manager', 'real', 'having', 'run-in', 'odds', 'last', 'no46', 'sisters', 'failed', 'term', 'course', 'where', 'who', 'she', 'reds', 'could', 'no83', 'by', 'taken', 'rankings', 'summer', 'can', 'those', 'stage', '1.5x', 'played', 'no2', 'sliders', 'highly', 'title', 'cup', 'her', 'contrast', 'began', 'champions', 'even', 'our', 'hubby', 'campaigns', 'perisic', 'me', 'returning', 'over', 'years', 'angelique', 'quality', 'avoided', 'seal', 'gave', 'really', 'hungry', 'months', 'champion', 'is', 'resignation', 'knock', 'enjoyed', 'pull', 'names

set에 저장된 데이터를 아래와 같은 형태의 딕셔너리로 만든다.  
{0: 'complain', 1: 'however', ..., 535: 'roma', 536: 'gruelling'}

In [4]:
# enumerate() 함수는 인수로 지정된 객체에 저장된 내용을 (인덱스, 데이터) 형태로 리턴한다.
# for index, word in enumerate(results):
    # print(index, word)
idx2word = dict(enumerate(results))
print(type(idx2word))
print(idx2word)

<class 'dict'>
{0: 'their', 1: 'sub-par', 2: 'wonderful', 3: 'tasted', 4: 'why', 5: 'dinner', 6: 'enjoy', 7: 'dominance', 8: 'courts', 9: 'waiter', 10: 'group', 11: 'year—and', 12: 'were', 13: 'illness', 14: 'clean', 15: 'serves', 16: 'commented', 17: 'look', 18: 'ai', 19: 'business', 20: 'victoria', 21: 'win', 22: 'quarter-final', 23: 'time', 24: 'although', 25: 'three', 26: 'a', 27: 'of', 28: 'halep', 29: 'dismal', 30: 'much', 31: 'than', 32: 'as', 33: 'once', 34: 'around', 35: 'hours', 36: 'some', 37: 'roma', 38: 'parking', 39: 'served', 40: 'places', 41: 'exhilarating', 42: 'if', 43: 'left', 44: 'rich', 45: 'didnt', 46: 'proved', 47: 'manager', 48: 'real', 49: 'having', 50: 'run-in', 51: 'odds', 52: 'last', 53: 'no46', 54: 'sisters', 55: 'failed', 56: 'term', 57: 'course', 58: 'where', 59: 'who', 60: 'she', 61: 'reds', 62: 'could', 63: 'no83', 64: 'by', 65: 'taken', 66: 'rankings', 67: 'summer', 68: 'can', 69: 'those', 70: 'stage', 71: '1.5x', 72: 'played', 73: 'no2', 74: 'sliders'

{0: 'complain', 1: 'however', ..., 535: 'roma', 536: 'gruelling'} 형태의 딕셔너리를 아래 형태의 딕셔너리로 변환한다.  
{'complain': 0, 'however': 1, ..., 'roma': 535, 'gruelling': 536}

In [5]:
# word2idx = {}
# for key in idx2word.keys():
    # print(key, idx2word[key])
    # word2idx[idx2word[key].strip()] = key
# for key, value in idx2word.items():
    # print(key, value)
    # word2idx[value.strip()] = key
word2idx = {value.strip(): key for key, value in idx2word.items()}
print(word2idx)

{'their': 0, 'sub-par': 1, 'wonderful': 2, 'tasted': 3, 'why': 4, 'dinner': 5, 'enjoy': 6, 'dominance': 7, 'courts': 8, 'waiter': 9, 'group': 10, 'year—and': 11, 'were': 12, 'illness': 13, 'clean': 14, 'serves': 15, 'commented': 16, 'look': 17, 'ai': 18, 'business': 19, 'victoria': 20, 'win': 21, 'quarter-final': 22, 'time': 23, 'although': 24, 'three': 25, 'a': 26, 'of': 27, 'halep': 28, 'dismal': 29, 'much': 30, 'than': 31, 'as': 32, 'once': 33, 'around': 34, 'hours': 35, 'some': 36, 'roma': 37, 'parking': 38, 'served': 39, 'places': 40, 'exhilarating': 41, 'if': 42, 'left': 43, 'rich': 44, 'didnt': 45, 'proved': 46, 'manager': 47, 'real': 48, 'having': 49, 'run-in': 50, 'odds': 51, 'last': 52, 'no46': 53, 'sisters': 54, 'failed': 55, 'term': 56, 'course': 57, 'where': 58, 'who': 59, 'she': 60, 'reds': 61, 'could': 62, 'no83': 63, 'by': 64, 'taken': 65, 'rankings': 66, 'summer': 67, 'can': 68, 'those': 69, 'stage': 70, '1.5x': 71, 'played': 72, 'no2': 73, 'sliders': 74, 'highly': 75,

word2idx 딕셔너리를 활용해서 모든 지문을 구성하는 단어 목록을 수치로 변환한 파생 변수를 데이터프레임에 추가한다.

In [6]:
# Dishplace is located in sunnyvale downtown ... => 46 288 137 399 54 233 ...
print(word2idx['dishplace'], word2idx['is'], word2idx['located'], word2idx['in'], word2idx['sunnyvale'], word2idx['downtown'])

194 100 217 175 184 495


In [7]:
# 지문을 구성하는 모든 단어를 수치로 변환하는 함수를 만든다.
def encode_paragraph(paragraph):
    # print(type(paragraph), paragraph)
    words = paragraph.split()
    # print(words)
    encoded = []
    for word in words:
        encoded.append([word2idx[word.lower()]])
    # print(encoded)
    return encoded

In [8]:
# 지문이 수치로 변환된 결과를 데이터프레임에 파생 변수로 추가한다. => [[46], [288], [137], [399], [54], [233], ...]
df['enc_paragraph'] = df.paragraph.apply(encode_paragraph)
df

,paragraph,category,enc_paragraph
0,Dishplace is located in sunnyvale downtown the...,food,"[[194], [100], [217], [175], [184], [495], [46..."
1,Sevice can be slower during busy hours but our...,food,"[[268], [68], [230], [309], [520], [252], [35]..."
2,Portions are huge both french toast and their ...,food,"[[314], [270], [421], [336], [196], [360], [11..."
3,We started with apps going the chicken and waf...,food,"[[278], [515], [514], [504], [393], [395], [26..."
4,The biscuits and gravy was too salty two peopl...,food,"[[395], [274], [117], [355], [503], [242], [40..."
5,The garlic fries were a great starter (and a h...,food,"[[395], [189], [496], [12], [26], [266], [534]..."
6,Our meal was excellent i had the pasta ai form...,food,"[[83], [302], [503], [272], [455], [447], [395..."
7,What i enjoy most about palo alto is so many r...,food,"[[286], [455], [6], [229], [239], [417], [161]..."
8,The drinks came out fairly quickly a good two ...,food,"[[395], [206], [334], [327], [321], [295], [26..."
9,Despite the not so good burger the service was...,food,"[[369], [395], [320], [373], [377], [294], [39..."


분류 항목(food[1, 0], sports[0, 1])을 원-핫 인코딩으로 수치로 변환한 파생 변수를 데이터프레임에 추가한다.

In [9]:
# 분류 항목을 원-핫 인코딩 수치로 변환하는 함수를 만든다.
def encode_category(category):
    # print(category)
    if category == 'food':
        return [1, 0]
    else:
        return [0, 1]

In [10]:
# 분류 항목이 수치로 변환된 결과를 데이터프레임에 파생 변수로 추가한다.
df['enc_category'] = df.category.apply(encode_category)
df

,paragraph,category,enc_paragraph,enc_category
0,Dishplace is located in sunnyvale downtown the...,food,"[[194], [100], [217], [175], [184], [495], [46...","[1, 0]"
1,Sevice can be slower during busy hours but our...,food,"[[268], [68], [230], [309], [520], [252], [35]...","[1, 0]"
2,Portions are huge both french toast and their ...,food,"[[314], [270], [421], [336], [196], [360], [11...","[1, 0]"
3,We started with apps going the chicken and waf...,food,"[[278], [515], [514], [504], [393], [395], [26...","[1, 0]"
4,The biscuits and gravy was too salty two peopl...,food,"[[395], [274], [117], [355], [503], [242], [40...","[1, 0]"
5,The garlic fries were a great starter (and a h...,food,"[[395], [189], [496], [12], [26], [266], [534]...","[1, 0]"
6,Our meal was excellent i had the pasta ai form...,food,"[[83], [302], [503], [272], [455], [447], [395...","[1, 0]"
7,What i enjoy most about palo alto is so many r...,food,"[[286], [455], [6], [229], [239], [417], [161]...","[1, 0]"
8,The drinks came out fairly quickly a good two ...,food,"[[395], [206], [334], [327], [321], [295], [26...","[1, 0]"
9,Despite the not so good burger the service was...,food,"[[369], [395], [320], [373], [377], [294], [39...","[1, 0]"


각 입력값의 실제 길이(단어 개수)를 알 수 있도록 각 지문별 단어의 개수를 파생 변수로 데이터프레임에 추가한다.

In [11]:
# 지문을 구성하는 단어의 개수를 세는 함수를 만든다.
def word_cnt(paragraph):
    return len(paragraph.split())

In [12]:
# 지문을 구성하는 단어의 게수를 데이터프레임에 파생 변수로 추가한다.
df['seq_length'] = df.paragraph.apply(word_cnt)
df

,paragraph,category,enc_paragraph,enc_category,seq_length
0,Dishplace is located in sunnyvale downtown the...,food,"[[194], [100], [217], [175], [184], [495], [46...","[1, 0]",53
1,Sevice can be slower during busy hours but our...,food,"[[268], [68], [230], [309], [520], [252], [35]...","[1, 0]",19
2,Portions are huge both french toast and their ...,food,"[[314], [270], [421], [336], [196], [360], [11...","[1, 0]",42
3,We started with apps going the chicken and waf...,food,"[[278], [515], [514], [504], [393], [395], [26...","[1, 0]",43
4,The biscuits and gravy was too salty two peopl...,food,"[[395], [274], [117], [355], [503], [242], [40...","[1, 0]",82
5,The garlic fries were a great starter (and a h...,food,"[[395], [189], [496], [12], [26], [266], [534]...","[1, 0]",24
6,Our meal was excellent i had the pasta ai form...,food,"[[83], [302], [503], [272], [455], [447], [395...","[1, 0]",50
7,What i enjoy most about palo alto is so many r...,food,"[[286], [455], [6], [229], [239], [417], [161]...","[1, 0]",43
8,The drinks came out fairly quickly a good two ...,food,"[[395], [206], [334], [327], [321], [295], [26...","[1, 0]",49
9,Despite the not so good burger the service was...,food,"[[369], [395], [320], [373], [377], [294], [39...","[1, 0]",82


RNN은 항상 같은 길이의 시퀀스를 받아야 한다.  
길이가 작은 입력 시퀀스는 패딩을 추가적으로 넣어서 모든 시퀀스 길이를 동일하게 설정한다.  
패딩이 RNN 계산에 영항을 끼치지 않도록 패딩 이전의 입력 시퀀시의 실제 길이를 파라미터로 받아 상태값 계산시 패딩을 제외한다.

In [13]:
# 최고로 긴 지문의 단어 수를 구한 후 모든 지문에 패딩을 집어넣어 최고로 긴 지문과 동일한 길이를 갖도록 만들기 위해
# 최고로 긴 지문의 단어수를 계산한다.
# max_word_cnt = 0
# for row in df.paragraph:
    # print(type(row), row)
    # if len(row.split()) > max_word_cnt:
        # max_word_cnt = len(row.split())
# for seq in df.seq_length:
    # if seq > max_word_cnt:
        # max_word_cnt = seq
max_word_cnt = df.seq_length.max()
print(max_word_cnt)

91


In [14]:
# 모든 지문에 패딩([-1])을 집어넣어 최고로 긴 문장과 동일한 길이를 갖게하는 함수를 만든다.
def sequence_padding(enc_paragraph):
    # print(type(enc_paragraph), len(enc_paragraph), enc_paragraph)
    for i in range(len(enc_paragraph), max_word_cnt):
        enc_paragraph.append([-1])
    return enc_paragraph

In [15]:
# 모든 지문에 패딩이 적용된 결과를 enc_paragraph 컬럼에 적용한다.
df['enc_paragraph'] = df.enc_paragraph.apply(sequence_padding)
print(len(df.enc_paragraph[1]))
print(df.enc_paragraph[1])
print(len(df.enc_paragraph[12]))
print(df.enc_paragraph[12])

91
[[268], [68], [230], [309], [520], [252], [35], [383], [83], [9], [503], [388], [117], [317], [95], [36], [266], [249], [209], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1]]
91
[[459], [238], [78], [379], [210], [477], [224], [175], [246], [357], [340], [21], [78], [414], [378], [527], [52], [11], [344], [235], [63], [60], [245], [435], [46], [286], [26], [216], [23], [258], [60], [100], [472], [395], [208], [76], [348], [185], [152], [378], [81], [402], [341], [395], [240], [358], [395], [196], [301], [210], [233], [528], [197], [391], [8], [60], [124], [340], [395], [240], [175], [476], [436], [235], [4

모델에서 사용할 입력값, 실제값, 지문의 실제 길이 파라미터를 만든다.

In [16]:
# 넘파이의 array() 메소드는 인수로 파이썬의 리스트나 튜플 데이터를 받아서 넘파이 배열로 변한하는 기능을 하기 때문에
# 데이터프레임의 시리즈를 바로 넣어주면 안되고 tolist() 메소드를 실행해서 파이썬의 리스트 형태로 변환한 후 넣어준다.
# print(type(df.enc_paragraph)) # <class 'pandas.core.series.Series'>
# print(type(df.enc_paragraph.tolist())) # <class 'list'>
enc_paragraph = np.array(df.enc_paragraph.tolist()) # 입력값
# print(type(enc_paragraph)) # <class 'numpy.ndarray'>
# print(enc_paragraph)
enc_category = np.array(df.enc_category.tolist()) # 실제값, 레이블
# print(enc_category)
seq_length = np.array(df.seq_length.tolist()) # 지문의 실제 길이
# print(seq_length)

x_train = enc_paragraph
y_train = enc_category
print(x_train.shape, y_train.shape)

(20, 91, 1) (20, 2)


지문을 읽고 주제를 분류하는 LSTM 모델을 만든다.

문맥 벡터(contextualized vector) 생성 단계

인덱스를 임베딩으로 변환한다. 임베딩은 학습 과정을 통해 단어 유사도를 포함하게 되어 문맥 벡터를 생성하는데 도움을 준다.  
인간의 언어(자연어)는 수치화되어있지 않은 데이터이기 때문에 머신러닝, 딥러닝 기법을 바로 사용할 수 없다. 그래서 자연어 처리에서 특징을 추출해서 수치화를 해줘야 하는데 이때 사용하는 것이 '언어의 벡터화'이다. 이런 벡터화 과정을 word embedding 이라고 한다.  
LSTM에 임베딩된 시퀀스를 입력해서 최종 상태값을 출력한다. => 최종 상태값이 문맥 벡터이다.

주제(food, sports) 분류 단계

문백 벡터를 덴즈 레이어에 입력하고 출려값을 노드가 2개인 덴즈 레이어에 입력한 후 노드가 2개인 덴즈 레이어의 출력값을 소프트 맥스에 입력시켜 food, sports에 대한 예측값을 계산한다.

<img src="./LSTM3.png" align="left" width="1000"/>

LSTM 모델을 구현한다.

In [17]:
tf.set_random_seed(35)
# 입력값과 실제값을 저장할 placeholder를 만든다.
# 입력값은 문장을 구성하는 단어들의 인덱스이며 그 길이는 지문을 구성하는 단어의 최대 개수(max_word_cnt)이다.
X = tf.placeholder(dtype=tf.float32, shape=[None, max_word_cnt, 1]) # 입력값을 기억할 placeholder, (20, 91, 1)
Y = tf.placeholder(dtype=tf.float32, shape=[None, 2]) # 실제값을 기억할 placeholder, (20, 2)

# 임베딩 레이어는 입력값(단어들의 인덱스)을 입력받아 5차원의 벡터 임베딩을 출력한다.
# layers.dense() 메소드는 densely connect layer 즉, '완전 연결 계층'을 만들어 준다.
embedding = tf.layers.dense(X, 5)
# LSTM 셀은 64차원의 벡터의 생성값을 출력한다.
cell = tf.nn.rnn_cell.LSTMCell(num_units=64)
# LSTM 셀의 출력값과 상태값을 저장한다.
outputs, state = tf.nn.dynamic_rnn(cell=cell, dtype=tf.float32, inputs=embedding, sequence_length=seq_length)

# 주제 분류는 두 개의 덴즈 레이어를 사용한다.
# 첫 번째 덴즈 레이어는 32개 노드를 가지고 있고 두 번째 덴즈 레이어는 2개의 노드를 가지고 있으며 이 2개의 노드가
# 소프트 맥스의 입력으로 들어간다.
dense_layer = tf.layers.dense(state.h, 32)
# logit은 food, sports를 원-핫 인코딩으로 구분하기 위해 2차원 벡터로 구성한다.
logit = tf.layers.dense(dense_layer, 2) # 최종 예측값

# 손실 함수 - 크로스 엔트로피
loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits_v2(logits=logit, labels=Y)
)
# Adam 옵티마이저
train = tf.train.AdamOptimizer(0.001).minimize(loss)

Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


모델 요약

In [18]:
# 문맥 벡터 생성
# 입력값은 단어들의 인덱스이며 그 길이는 항상 91이다.
print(X) # Tensor("Placeholder:0", shape=(?, 91, 1), dtype=float32)
# 임베딩 레이어는 인덱스를 받아 5차원 벡터의 임베딩을 출력한다.
print(embedding) # Tensor("dense/BiasAdd:0", shape=(?, 91, 5), dtype=float32)
# LSTM 셀은 64차원의 상태값을 출력한다.
print(state.h) # Tensor("rnn/while/Exit_4:0", shape=(?, 64), dtype=float32)

# 문맥 벡터를 사용해 지문의 주제 분류하기, 주제 분류는 2개의 덴즈 레이어를 사용했다.
# 첫 번째 덴즈 레이어는 32개의 노드를 가진다.
print(dense_layer) # Tensor("dense_1/BiasAdd:0", shape=(?, 32), dtype=float32)
# 두 번째 덴즈 레이어는 2개의 노드를 가지고 있으며, 이 2개의 노드는 소프트 맥스의 입력으로 들어간다.
print(logit) # Tensor("dense_2/BiasAdd:0", shape=(?, 2), dtype=float32)

Tensor("Placeholder:0", shape=(?, 91, 1), dtype=float32)
Tensor("dense/BiasAdd:0", shape=(?, 91, 5), dtype=float32)
Tensor("rnn/while/Exit_4:0", shape=(?, 64), dtype=float32)
Tensor("dense_1/BiasAdd:0", shape=(?, 32), dtype=float32)
Tensor("dense_2/BiasAdd:0", shape=(?, 2), dtype=float32)


학습 시킨다.

In [19]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(401):
        _, loss_ = sess.run([train, loss], feed_dict={X: x_train, Y: y_train})
        if epoch % 20 == 0:
            predict = tf.nn.softmax(logit) # 최종 예측값
            # 정확도를 계산하는 수식을 만든다.
            correct_predict = tf.equal(tf.argmax(predict, 1), tf.argmax(Y, 1))
            accuracy = tf.reduce_mean(tf.cast(correct_predict, 'float'))
            current_accuracy = accuracy.eval(feed_dict={X: x_train, Y: y_train}) # 정확도 계산
            print('epoch: {:3d}, loss: {:6.4f}, accuracy: {:6.4f}'.format(epoch, loss_, current_accuracy))

epoch:   0, loss: 0.7324, accuracy: 0.5000
epoch:  20, loss: 0.5984, accuracy: 0.7500
epoch:  40, loss: 0.4820, accuracy: 0.8500
epoch:  60, loss: 0.3112, accuracy: 0.8500
epoch:  80, loss: 0.1436, accuracy: 0.9500
epoch: 100, loss: 0.0563, accuracy: 1.0000
epoch: 120, loss: 0.0183, accuracy: 1.0000
epoch: 140, loss: 0.0066, accuracy: 1.0000
epoch: 160, loss: 0.0031, accuracy: 1.0000
epoch: 180, loss: 0.0018, accuracy: 1.0000
epoch: 200, loss: 0.0012, accuracy: 1.0000
epoch: 220, loss: 0.0009, accuracy: 1.0000
epoch: 240, loss: 0.0006, accuracy: 1.0000
epoch: 260, loss: 0.0005, accuracy: 1.0000
epoch: 280, loss: 0.0004, accuracy: 1.0000
epoch: 300, loss: 0.0003, accuracy: 1.0000
epoch: 320, loss: 0.0002, accuracy: 1.0000
epoch: 340, loss: 0.0002, accuracy: 1.0000
epoch: 360, loss: 0.0002, accuracy: 1.0000
epoch: 380, loss: 0.0001, accuracy: 1.0000
epoch: 400, loss: 0.0001, accuracy: 1.0000
